In [1]:
# created on Dec 24, 2020
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [7]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [6]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "kCXIRLrxgLVIVcC4mecCDn3cz"
    consumer_secret = "yzuvyJPZ6hveorPAPsO3ELExqr8yXxWUjrkqIkclOtE4bvbG6w"
    access_token = "2605784318-1SuOqdOAwuklXbdaBi2khH5CULkbyBvsYKFgIoZ"
    access_token_secret = "kMNZQkQAu4gewtpMEHniuUw8lCkDEz7nvTWjnGE35B9ZH"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [98.938481, 19.531519, 124.464988, 48.874815] 

    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)

{'created_at': 'Fri Jan 29 06:49:56 +0000 2021', 'id': 1355045497958883331, 'id_str': '1355045497958883331', 'text': 'やっば、この時間からは早すぎるのかな（笑）\n\nアニメ特撮バーGは開いております❤️', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3329544799, 'id_str': '3329544799', 'name': '前田シゲ@アニメ特撮バーＧ 熊本ウイイレ好き', 'screen_name': 'mihokugashige', 'location': '熊本市中央区花畑町11-33RS-2ビル３階', 'url': 'https://www.instagram.com/maedashige/', 'description': 'アニメ特撮バーG１９時〜朝４時まで営業中。イベント&結婚式の司会事務所 http://SG.officeも宜しく！！ #前田シゲのノリノリドライビング #パーラーナポレオンcm出演中', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 823, 'friends_count': 367, 'listed_count': 7, 'favourites_count': 1781, 'statuses_count': 2256, 'created_at': 'Mon Aug 24 19:21:04 +0000 2015', 'utc_offset': Non

{'created_at': 'Fri Jan 29 06:50:09 +0000 2021', 'id': 1355045549578145793, 'id_str': '1355045549578145793', 'text': 'せやねん。\n\nそーいや、しぞーかの清水でその昔、アメリカへの出稼ぎが多かったのは砂地で耕作の困難な三保の人やった。', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 312523703, 'id_str': '312523703', 'name': 'ノラ神主ぴーちゃん 🍵反緊縮 Pete the Stray Shinto Priest', 'screen_name': 'PeterYokoyama', 'location': '大阪上本町・岡山神道山・静岡草薙・中国香港', 'url': None, 'description': 'ただのアホになりたい☆「いいね」はお祈りの印なことが多いです☆新興宗教関係者(神道・黒住教のノラ神主・訓導)☆Priest of Kurozumi Shinto☆ユングとケインズ好き☆令和元年に行政書士試験合格☆福島・相馬に逃れた諏訪氏の末裔☆牧瀬里穂似のオッサン☆嫁はんはลาวไทย☆香港市民', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 2016, 'friends_count': 2538, 'listed_count': 34, 'favourites_count': 176633, 'statuses_count': 129208, 

{'created_at': 'Fri Jan 29 06:50:15 +0000 2021', 'id': 1355045577646460931, 'id_str': '1355045577646460931', 'text': 'Эрүүл мэндийн сайд Энхболд гэж хэнийн 🙄🙄🙄', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1160421787303198730, 'id_str': '1160421787303198730', 'name': 'Khulan Shinejil➿', 'screen_name': 'khulan9009', 'location': 'Mongolia', 'url': None, 'description': 'Монгол Улсын Иргэн 🇲🇳 Шинэхэн жиргээч😁', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 8460, 'friends_count': 6793, 'listed_count': 3, 'favourites_count': 18156, 'statuses_count': 14136, 'created_at': 'Sun Aug 11 05:24:48 +0000 2019', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is

{'created_at': 'Fri Jan 29 06:50:18 +0000 2021', 'id': 1355045590346829828, 'id_str': '1355045590346829828', 'text': '@Jack32397140 @Rengongmeng 第二句看不懂', 'display_text_range': [27, 33], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1355033636127424516, 'in_reply_to_status_id_str': '1355033636127424516', 'in_reply_to_user_id': 1185464115763920896, 'in_reply_to_user_id_str': '1185464115763920896', 'in_reply_to_screen_name': 'Jack32397140', 'user': {'id': 1352451265570041857, 'id_str': '1352451265570041857', 'name': 'Sally', 'screen_name': 'sally6792', 'location': None, 'url': None, 'description': '热爱身心灵类，热爱传统文化，热爱所有的美好', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 6, 'friends_count': 25, 'listed_count': 0, 'favourites_count': 54, 'statuses_count': 49, 'created_at': 'Fri Jan 22 03:01:35 +0000 2021', 'utc_offset': None, 'time_zone': None, 'geo_enabled': Tr

{'created_at': 'Fri Jan 29 06:50:28 +0000 2021', 'id': 1355045629186117634, 'id_str': '1355045629186117634', 'text': '@RadioEdith Get you.', 'display_text_range': [12, 20], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1355036030576054273, 'in_reply_to_status_id_str': '1355036030576054273', 'in_reply_to_user_id': 76629787, 'in_reply_to_user_id_str': '76629787', 'in_reply_to_screen_name': 'RadioEdith', 'user': {'id': 297338634, 'id_str': '297338634', 'name': 'Nomfundo 💀', 'screen_name': 'Nomfundo_PM', 'location': 'Ladysmith 🇿🇦, Shanghai 🇨🇳', 'url': None, 'description': 'I am returning to HER 👼🏾', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1336, 'friends_count': 1228, 'listed_count': 4, 'favourites_count': 34725, 'statuses_count': 25273, 'created_at': 'Thu May 12 10:39:26 +0000 2011', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': N

...

{'created_at': 'Fri Jan 29 06:53:50 +0000 2021', 'id': 1355046478037716993, 'id_str': '1355046478037716993', 'text': '@yangtheonly 那妳也會滿出來喔，壞笑', 'display_text_range': [13, 24], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1354818337377644547, 'in_reply_to_status_id_str': '1354818337377644547', 'in_reply_to_user_id': 1224559178724700160, 'in_reply_to_user_id_str': '1224559178724700160', 'in_reply_to_screen_name': 'yangtheonly', 'user': {'id': 4858729029, 'id_str': '4858729029', 'name': '雜草蘇偉列車長', 'screen_name': 'SuwiiLin', 'location': 'New Taipei City, Taiwan', 'url': None, 'description': '沒有什麼不可以，\n只有做了才知道行不行。         \n沒有什麼先緩緩，\n只有試了才知道能不能。', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1084, 'friends_count': 228, 'listed_count': 2, 'favourites_count': 19091, 'statuses_count': 6008, 'created_at': 'Fri Jan 29 03:10:25 +0000 2016', 'utc_offset': None,

{'created_at': 'Fri Jan 29 06:53:54 +0000 2021', 'id': 1355046493091106818, 'id_str': '1355046493091106818', 'text': 'Buti naka-mask hahahaha\n \nREQUEST @SB19Official @MTV #FridayLivestream\n\n#SB19Memeories\n#SB19', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1257581505536126976, 'id_str': '1257581505536126976', 'name': 'Valerie 🇭🇰🇵🇭 | Vote and stream now!', 'screen_name': 'EirelavTin', 'location': '🇭🇰 x 🇵🇭', 'url': 'https://atinxsb19.com', 'description': 'Fan acct. OT5 💙 Simple pero rock.\nA reader and a writer. \nMain acct: @mangkunifroggy', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 772, 'friends_count': 789, 'listed_count': 2, 'favourites_count': 16242, 'statuses_count': 19540, 'created_at': '

{'created_at': 'Fri Jan 29 06:54:00 +0000 2021', 'id': 1355046518403698688, 'id_str': '1355046518403698688', 'text': '泡澡是这个世界上最享受的事情 https://t.co/AGWA2h4nV5', 'display_text_range': [0, 14], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1318476954782244864, 'id_str': '1318476954782244864', 'name': 'wting', 'screen_name': 'wtingggggg', 'location': None, 'url': None, 'description': 'd', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 3, 'friends_count': 3, 'listed_count': 0, 'favourites_count': 0, 'statuses_count': 4, 'created_at': 'Tue Oct 20 08:59:48 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background

{'created_at': 'Fri Jan 29 06:54:30 +0000 2021', 'id': 1355046643918225411, 'id_str': '1355046643918225411', 'text': 'Desayuno de campeón #chuchoenchina #guangzhou #china @chuchoenchina #pekinpato #campeon https://t.co/Q0TUptcjPU', 'display_text_range': [0, 87], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1347674297238646785, 'id_str': '1347674297238646785', 'name': 'chuchoenchina', 'screen_name': 'chuchoenchina', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 31, 'friends_count': 113, 'listed_count': 0, 'favourites_count': 5, 'statuses_count': 22, 'created_at': 'Fri Jan 08 22:39:41 +0000 2021', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True,

{'created_at': 'Fri Jan 29 06:54:45 +0000 2021', 'id': 1355046707881369604, 'id_str': '1355046707881369604', 'text': '你们喜欢这样的身材🐔吗@wusaile1 https://t.co/g2nsWhQUUr', 'display_text_range': [0, 20], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1322483782511816704, 'id_str': '1322483782511816704', 'name': '脩先生一韓先生', 'screen_name': 'wusaile1', 'location': None, 'url': None, 'description': '25.178.63.19）（双）江苏，上海，南京不定时飞，蜜桃臀，香蕉diao上翘，肌肉男，可v：HDD1158', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1946, 'friends_count': 45, 'listed_count': 5, 'favourites_count': 50, 'statuses_count': 31, 'created_at': 'Sat Oct 31 10:21:38 +0000 2020', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'cont